# SCADA Flow Rate Calculation Model

In [56]:
import pandas as pd
import os
import csv
import math

import re
import datetime
import io
from zipfile import ZipFile

import pandas as pd
import numpy as np
import pandas_gbq
import janitor

# Section 1

In this section, I read my Lift Station SCADA data and Lift Station Legend data to a pandas data frames and do some quick formatting. I'll begin by converting the date and time column to DateTime format, dropping unneeded columns, and dropping any NA values. Once this process is finished we are ready to begin section 2.

## Read SCADA data into Pandas Dataframe

In [57]:
df = pd.read_csv('LiftStation.csv')

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2210235391.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('LiftStation.csv')


In [58]:
legend = pd.read_csv('Lift Station Legend Cross Referance.csv')

In [59]:
legend = legend.drop('Pump #1 Run', axis=1)
legend = legend.drop('Pump #2 Run', axis=1)
legend = legend.drop('Pump #3 Run', axis=1)
legend = legend.drop('Table Tag #.4', axis=1)
legend = legend.drop('Pump #4 Run', axis=1)
legend = legend.drop('Unnamed: 11', axis=1)

## Convert the DateAndTime column to Datetime Format

In [60]:
df['PumpStart'] = pd.to_datetime(df['PumpStart'])
df['PumpStop'] = pd.to_datetime(df['PumpStop'])

## Format the dataframe

In [61]:
df = df.dropna(subset=['PumpStart', 'PumpStop'])

In [62]:
df['PumpTagIndex'] = df['PumpTagIndex'].astype(int)

In [63]:
# create a boolean mask of rows with duplicate "PumpTagIndex" and "PumpStart" values
mask = df.duplicated(subset =['PumpTagIndex', 'PumpStart'], keep = 'first')

# drop the duplicate rows using the boolean mask

#now let's drop the duplicate rows
new_df = df[~mask]
# reset the index after dropping rows
new_df.reset_index(drop=True, inplace=True)

# print the length of the new dataframe
#print(len(new_df))

In [64]:
# create a boolean mask of rows with duplicate "PumpTagIndex" and "PumpStop" values
mask = new_df.duplicated(subset=['PumpTagIndex', 'PumpStop'], keep='first')

# drop the duplicate rows using the boolean mask
new_df = new_df[~mask]

# reset the index after dropping rows
new_df.reset_index(drop=True, inplace=True)

# print the length of the new dataframe
#print(len(new_df))

# Section 2

In this section, I will create a new data frame called calc_df, which will hold all of our calculated data going forward. 

I'll begin by assigning the original df to my calculated df, and reformatting the columns (names and order). 

Next I will remove any erroneous rows containing invalid pump numbers.

Then I'll use the legend dataframe I read into the program earlier to add the correct lift station name to each row in calc_df.

Finally, I will convert both on and off timestamps into Unix time and calculate the difference in time between the pump turning on and off. 

Note: Remove the # in front of any of the calc_df lines to see a preview of the data frame.

## Create new data frame

In [65]:
calc_df = new_df
calc_df = calc_df.rename(columns={'PumpStart': 'PumpStartTime'})
calc_df = calc_df.rename(columns={'PumpStop': 'PumpStopTime'})
calc_df = calc_df.rename(columns={'WellStart': 'StartWellLevel'})
calc_df = calc_df.rename(columns={'WellEnd': 'StopWellLevel'})

In [66]:
calc_df['LiftStation'] = None
calc_df['StartUnix'] = 0.0
calc_df['StopUnix'] = 0.0
calc_df['Time_diff'] = 0.0

In [67]:
calc_df = calc_df[['LiftStation','PumpTagIndex', 'PumpStartTime', 'StartWellLevel','PumpStopTime', 'StopWellLevel', 'StartUnix', 'StopUnix','Time_diff']]

In [68]:
#calc_df

## Delete any rows with incorrect pump numbers

In [69]:
#create a list of valid pump numbers

valid_pumps = []

for index, row in legend.iterrows():
    valid_pumps.append(row['Table Tag #'])
    valid_pumps.append(row['Table Tag #.1'])
    valid_pumps.append(row['Table Tag #.2'])
    valid_pumps.append(row['Table Tag #.3'])
    
valid_pumps = list(filter(lambda x: not math.isnan(x), valid_pumps))
        
#len(valid_pumps)

In [70]:
calc_df = calc_df[calc_df['PumpTagIndex'].isin(valid_pumps)]
calc_df = calc_df.reset_index(drop=True)
calc_df['PumpTagIndex'] = calc_df['PumpTagIndex'].astype(int)

## Add the correct Lift Station to each row in calc_df

Here I am going to perfom 4 joins between the legend dataframe and my calc_df. Each join will merge the legend and calc data frames on one of the 4 table tag #'s.

Table Tag 0 Join

In [71]:
# Left join calc_df to legend df where Pump Tag Index = Table Tag
merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #', how='left')

# Create a mask to filter the rows where Lift Station is not empty
mask = merged_df['Lift Station'].notnull()

# Assign the Lift Station values to the LiftStation column in the calc_df dataframe
calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']

Table Tag 1 Join

In [72]:
merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #.1', how='left')

# Create a mask to filter the rows where Lift Station is not empty
mask = merged_df['Lift Station'].notnull()

# Assign the Lift Station values to the LiftStation column in the calc_df dataframe
calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']

Table Tag 2 Join

In [73]:
merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #.2', how='left')

# Create a mask to filter the rows where Lift Station is not empty
mask = merged_df['Lift Station'].notnull()

# Assign the Lift Station values to the LiftStation column in the calc_df dataframe
calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']

Table Tag 3 Join

In [74]:
merged_df = calc_df.merge(legend, left_on='PumpTagIndex', right_on='Table Tag #.3', how='left')

# Create a mask to filter the rows where Lift Station is not empty
mask = merged_df['Lift Station'].notnull()

# Assign the Lift Station values to the LiftStation column in the calc_df dataframe
calc_df.loc[mask, 'LiftStation'] = merged_df.loc[mask, 'Lift Station']

Remove unwanted lift stations

In [75]:
calc_df = calc_df.drop(calc_df[calc_df['LiftStation'].isin(['Stratford 2662 (20)', 'Linda Vista Lift Station (5)', 'Linda Vista Golf Course (12)', 'Reserve Street (14)'])].index)
calc_df = calc_df.reset_index(drop=True)

## Now let's get Unix time for each timestamp

In [76]:
calc_df['StartUnix'] = pd.to_datetime(calc_df['PumpStartTime']).astype(int)/ 10**9
calc_df['StopUnix'] = pd.to_datetime(calc_df['PumpStopTime']).astype(int) / 10**9

In [77]:
#calc_df.head(20)

## Calculating the time difference between Pumps turning on and off

In [78]:
calc_df['Time_diff'] = (calc_df['StopUnix'] - calc_df['StartUnix']) / 60

# Section 3

In this section, I will add elevation difference, volume per foot, observed pumping flow rate, the average flow rate per pump, and average observed pumping flow rate (GPM).

Elevation Difference - This value can be changed for the desired lift station. Please note that each lift station will have a different elevation level. To change the elevation for a different lift station, simply change the 2.2 in the first cell below.

Observed pumping flow rate - To calculate this value we first need to input the wet wells diameter, again this value will differ per each lift station. Simply change the diameter in the diam_well = (5) statement to the desired value. From there the vol_per_foot calculation will update and be added to the calc_df. Now we have all the information in the calc_df to calculate the observed pumping flow rate. I'll take the elevation difference value times the volume per foot value, and divide the result by our time difference. The resulting value will be added to the calc_df in the observed pumping flow rate column.

Average flow rate per pump - Average flow rate per pump is calculated by adding the values in the observed pumping flow rate column for each pump and then dividing that sum by the count of values for that pump. So if I iterate over 4 rows of the calc_df, and pump 1 has 3 out of the 4 rows with an observed pumping flow rate of 100, 120, and 100, the formula will sum these three rows (320) and divide by the number of rows that were related to pump 1 (3). The average flow rate per pump 1 in row 4 would be 106.

Average observed pumping flow rate (GPM) - to calculate this value I simply add all the average flow rates for both pumps and divide the sum by the count of rows I have iterated over. For example, if I have iterated over 4 rows, and 2 of the rows are pump 1 and 2 are pump 2, I'll add up the values from both pumps (100,110,100,120) and divide the sum by 4 to calculate the average observed pumping flow rate in GPM of 107.5 in row 4.



## Add in Elevation Difference

In [79]:
elevation_dict = {}

for index,row in legend.iterrows():
    tag = row['Lift Station']
    elevation_dict[tag] = row['Elevation Diff']

for index, row in calc_df.iterrows():
    tag = row['LiftStation']
    calc_df.at[index,'Elevation_dif'] = elevation_dict[tag]

## Calculate Observed Pumping Flow Rate

### Calculate Volume Per Foot for each well

In [80]:
#initialize new column
calc_df['wet_well_diam'] = 0.0

# left join calc_df and legend df on lift station
merged_df = calc_df.merge(legend, left_on = 'LiftStation', right_on = 'Lift Station', how = 'left')

#create a boolean mask to find all rows where we have a wet well diameter
mask = merged_df['Wet Well Diam'].notnull()

#add the wet well diameter value from the legend dataframe to the calc_df where mask is true
calc_df.loc[mask, 'wet_well_diam'] = merged_df.loc[mask, 'Wet Well Diam']

In [81]:
#Calculate volume per foot (check 4*7.48)
calc_df['volume_per_foot'] = round(((calc_df['wet_well_diam']*calc_df['wet_well_diam']) * 3.14)/4*7.48)

### Calculate Observed Pumping Flow Rate

In [82]:
calc_df['observed_pumping_flow_rate'] = int()

In [83]:
for index, row in calc_df.iterrows():
    result = round((row['Elevation_dif'] * row['volume_per_foot'])/row['Time_diff'])
    calc_df.at[index,'observed_pumping_flow_rate'] = result

In [84]:
#calc_df

## Calculate Average Flow Rate Per Pump

In [85]:
calc_df['avg_flow_rate_pump'] = int()

### Create two dictionaries
### 1. Dictionary containing a running total of Observed Pumping Flow Rate for each pump
### 2. Dictionary containing a running row count of Observed Pumping Flow Rate for each pump

In [86]:
sum_dict ={}
count_dict ={}

for item in valid_pumps:
    result = 0
    sum_dict[item] = result
    count_dict[item] = result

### Calculate Average Flow Rate Per Pump

In [87]:
for index, row in calc_df.iterrows():
    tag = row["PumpTagIndex"]
    sum_dict[tag] += row['observed_pumping_flow_rate']
    count_dict[tag] += 1
    calc_df.at[index, 'avg_flow_rate_pump'] = sum_dict[tag] / count_dict[tag]    

## Finally Let's calculate our Average Observed Pumping Flow Rate (GPM)

In [88]:
calc_df['average_observed_pumping_flow_rate'] = int()

### Create a dictionary for sum and count for each Lift Station

In [89]:
sum_dict ={}
count_dict ={}

for index, row in legend.iterrows():
    result = 0
    tag = row['Lift Station']
    sum_dict[tag] = result
    count_dict[tag] = result

### Calculate Average Observed Pumping Flow Rate (GPM)

In [90]:
for index, row in calc_df.iterrows():
    tag = row["LiftStation"]
    sum_dict[tag] += row['observed_pumping_flow_rate']
    count_dict[tag] += 1
    calc_df.at[index, 'average_observed_pumping_flow_rate'] = sum_dict[tag] / count_dict[tag]  

In [91]:
#filtered_df = calc_df[calc_df['LiftStation'] == 'Waldo (22)']
#filtered_df.head(60)

# Section 4

In this section, I will calculate the time to fill, inflow, and average inflow. These values need to be accounted for when calculating flow rate, as even when the pumps are turned on within a wet well and the level is dropping, water is still flowing through the sewer pipes into the well.

Time to fill - To calculate the time to fill for each row in calc_df, I take the stop Unix time of the first row within calc_df (when the pump turned off) and subtract this value from the start Unix time of the next row (when the pump turned back on). The difference between the pump turning off and the next pump turning on is my time to fill the measure.

Inflow - To calculate my inflow value for each row in calc_df, I take the row's volume per foot value and multiply it by the row's elevation difference value. I then divide that result by the row's time to fill value we just calculated.

Average Inflow - To calculate my average inflow for each row in calc_df, I simply add all the values for inflow in the rows I have iterated over and divide by the count of those rows. So if I have iterated over 4 rows so far in calc_df and the inflow values are (60,40,20,60) I sum these values (180) and divide by four to get my average inflow value in row 4, 45.


## Calculate time to fill

In [92]:
calc_df['Time_to_Fill'] = float

### Create two dictionaries
### 1. Dictionary containing the Start Unix time for each Lift Station
### 2. Dictionary containing the Stop Unix time for each Lift Station

In [93]:
# Sort the dataframe by LiftStation and PumpStartTime
calc_df = calc_df.sort_values(by=['LiftStation', 'PumpStartTime'])
calc_df = calc_df.reset_index(drop=True)

In [94]:
same_time_pumps = pd.DataFrame(columns=calc_df.columns)
rows_to_drop = []

for index, row in calc_df.iterrows():
    if index == 0:
        calc_df.at[index, 'Time_to_Fill'] = 0
        continue
    
    if row['LiftStation'] == calc_df.iloc[index-1]['LiftStation']:
        result = (row['StartUnix'] - calc_df.iloc[index-1]['StopUnix'])/60
        if result > 1.0:
            calc_df.at[index, 'Time_to_Fill'] = result
        else:
            rows_to_drop.append(index-1)
            rows_to_drop.append(index)
            same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
            same_time_pumps = same_time_pumps.append(calc_df.iloc[index])
        
    if row['LiftStation'] != calc_df.iloc[index-1]['LiftStation']:
        result = 0
        calc_df.at[index, 'Time_to_Fill'] = result

# Drop the rows to be removed from calc_df
calc_df.drop(index=rows_to_drop, inplace=True)
# Reset the index of calc_df after dropping rows
calc_df = calc_df.reset_index(drop=True)      

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pan

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pan

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pan

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pan

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pan

...

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from panda

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pan

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pan

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pan

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_time_pumps = same_time_pumps.append(calc_df.iloc[index-1])
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_887/2585980968.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pan

In [95]:
#negative_values = calc_df.loc[calc_df['Time_to_Fill'] < 1, 'Time_to_Fill']
#negative_values

## Calculate Inflow and Average Inflow

In [96]:
calc_df['Inflow'] = float()
calc_df['Avg_Inflow'] = float()

In [97]:
for index, row in calc_df.iterrows():
    if row['Time_to_Fill'] > 0:
        inflow = row['volume_per_foot'] * row['Elevation_dif'] / row['Time_to_Fill']
        calc_df.at[index, 'Inflow'] = inflow
    else:
        pass

In [98]:
sum_dict ={}
count_dict ={}

for index, row in legend.iterrows():
    result = 0
    tag = row['Lift Station']
    sum_dict[tag] = result
    count_dict[tag] = result
    
for index, row in calc_df.iterrows():
    if row['Time_to_Fill'] > 0:
        tag = row['LiftStation']
        sum_dict[tag] += row['Inflow']
        count_dict[tag] += 1
        calc_df.at[index, 'Avg_Inflow'] = sum_dict[tag] / count_dict[tag]


# Section 5

In this section, I will calculate the actual pump flow rate in GPM, the average flow rate per pump in GPM, and the average pump flow rate in GPM.

Actual pump flow rate - To calculate the actual pump flow rate for each row in calc_df, I simply take the row's observed pumping flow rate value we calculated earlier, and add the row's inflow value.

Average flow rate per pump - To calculate the average flow rate per pump I add the values in the actual pump flow rate column for each pump, and then divide that sum by the count of values for that pump. So if I iterate over 4 rows of the calc_df, and pump 1 has 3 out of the 4 rows with an actual pump flow rate of 100, 120, and 100, the formula will sum these three rows (320) and divide by the number of rows that were related to pump 1 (3). The average flow rate per pump 1 in row 4 would be 106.

Average pump flow rate - to calculate this value I simply add all the actual pump flow rates for both pumps and divide the sum by the count of rows I have iterated over. For example, if I have iterated over 4 rows, and 2 of the rows are pump 1 and 2 are pump 2, I'll add up the values from both pumps (100,110,100,120) and divide the sum by 4 to calculate the average pump flow rate in GPM of 107.5.

## Calculate Actual Pump Flow Rate (GPM)

In [99]:
calc_df['Actual_Pump_Flow_Rate'] = float()

In [100]:
for index, row in calc_df.iterrows():
    result = round(row['observed_pumping_flow_rate'] + row['Inflow'])
    calc_df.at[index, 'Actual_Pump_Flow_Rate'] = result

## Finally Let's calculate our Average Flow Rate Per Pump and the Average Pump Flow Rate in (GPM)

In [101]:
calc_df['Average_Pump_Flow_Rate'] = int()
calc_df['Average_Pump_Flow_Rate_(all)'] = int()

In [102]:
pump_sum_dict={}
pump_count_dict={}

for item in valid_pumps:
    pump_sum_dict[item]=0
    pump_count_dict[item]=0
    

for index, row in calc_df.iterrows():
    tag = row['PumpTagIndex']
    pump_sum_dict[tag] += row['Actual_Pump_Flow_Rate']
    pump_count_dict[tag] += 1
    calc_df.at[index,'Average_Pump_Flow_Rate'] = pump_sum_dict[tag] / pump_count_dict[tag]

In [103]:
sum_dict ={}
count_dict ={}

for index, row in legend.iterrows():
    result = 0
    tag = row['Lift Station']
    sum_dict[tag] = result
    count_dict[tag] = result

for index, row in calc_df.iterrows():
    tag = row['LiftStation']
    sum_dict[tag] += row['Actual_Pump_Flow_Rate']
    count_dict[tag] += 1
    calc_df.at[index,'Average_Pump_Flow_Rate_(all)'] = sum_dict[tag] / count_dict[tag]

In [105]:
calc_df

,LiftStation,PumpTagIndex,PumpStartTime,StartWellLevel,PumpStopTime,StopWellLevel,StartUnix,StopUnix,Time_diff,Elevation_dif,...,volume_per_foot,observed_pumping_flow_rate,avg_flow_rate_pump,average_observed_pumping_flow_rate,Time_to_Fill,Inflow,Avg_Inflow,Actual_Pump_Flow_Rate,Average_Pump_Flow_Rate,Average_Pump_Flow_Rate_(all)
0,Buena Vista (19),920,2022-06-24 00:11:33,3.200578,2022-06-24 00:13:32,2.189985,1.656029e+09,1.656030e+09,1.983333,1.0,...,211.0,106,106.000000,106.000000,0,0.000000,0.000000,106.0,106.000000,106.000000
1,Buena Vista (19),918,2022-06-24 03:33:31,3.196136,2022-06-24 03:35:31,2.187765,1.656042e+09,1.656042e+09,2.000000,1.0,...,211.0,106,106.000000,106.000000,199.983333,1.055088,1.055088,107.0,107.000000,106.500000
2,Buena Vista (19),920,2022-06-24 06:24:22,3.198357,2022-06-24 06:26:18,2.232186,1.656052e+09,1.656052e+09,1.933333,1.0,...,211.0,109,107.500000,107.000000,168.85,1.249630,1.152359,110.0,108.000000,107.666667
3,Buena Vista (19),918,2022-06-24 07:32:38,3.200578,2022-06-24 07:34:38,2.229965,1.656056e+09,1.656056e+09,2.000000,1.0,...,211.0,106,106.000000,106.750000,66.333333,3.180905,1.828541,109.0,108.000000,108.000000
4,Buena Vista (19),920,2022-06-24 08:49:01,3.196136,2022-06-24 08:50:59,2.192206,1.656061e+09,1.656061e+09,1.966667,1.0,...,211.0,107,107.333333,106.800000,74.383333,2.836657,2.080570,110.0,108.666667,108.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348288,Waldo (22),1033,2023-01-29 23:14:11,4.922175,2023-01-29 23:15:17,3.814686,1.675034e+09,1.675034e+09,1.100000,1.0,...,376.0,342,368.580268,380.361486,58.866667,6.387316,2.059379,348.0,345.712042,342.235450
348289,Waldo (22),1032,2023-01-30 08:24:31,4.919978,2023-01-30 08:25:38,3.812489,1.675067e+09,1.675067e+09,1.116667,1.0,...,376.0,337,392.289608,380.324895,549.233333,0.684591,2.058167,338.0,338.683274,342.231718
348290,Waldo (22),1033,2023-01-30 11:15:06,5.019593,2023-01-30 11:16:12,3.816151,1.675077e+09,1.675077e+09,1.100000,1.0,...,376.0,342,368.535893,380.292580,169.466667,2.218725,2.058308,344.0,345.709059,342.233275
348291,Waldo (22),1032,2023-01-30 17:08:39,4.926570,2023-01-30 17:09:47,3.818348,1.675099e+09,1.675099e+09,1.133333,1.0,...,376.0,332,392.187075,380.251896,352.45,1.066818,2.057435,333.0,338.673179,342.225154


## Export the data to CSV

In [39]:
calc_df.to_csv("Model Data for Kelly Lift Station.csv")

In [ ]:
same_time_pumps.to_csv("Pumps Running at the same time.csv")